In [1]:
!pip install scikit-learn
!pip install nltk
!pip install gensim
!pip install pyLDAvis
!pip install bertopic
!pip install umap-learn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import re
import nltk
import pickle
import numpy as np
import pyLDAvis
pyLDAvis.enable_notebook()

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from bertopic import BERTopic


In [3]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
dataset = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print("Total documents:", len(documents))


Total documents: 11314


In [5]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words and len(w) > 3]
    return " ".join(tokens)

cleaned_docs = [preprocess(doc) for doc in documents]
print("Sample preprocessed doc:\n", cleaned_docs[0][:200])


Sample preprocessed doc:
 wondering anyone could enlighten door sports looked late early called bricklin doors really small addition front bumper separate rest body know anyone tellme model name engine specs years production m


In [6]:
# For LDA → CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
dtm = count_vectorizer.fit_transform(cleaned_docs)

# For NMF → TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf = tfidf_vectorizer.fit_transform(cleaned_docs)


In [7]:
n_topics = 10

lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(dtm)
print("LDA Perplexity:", lda.perplexity(dtm))

# Document-topic distribution for pyLDAvis
doc_topic_dists = lda.transform(dtm)


LDA Perplexity: 5336.914279211712


In [8]:
nmf = NMF(n_components=n_topics, random_state=42)
nmf.fit(tfidf)


,n_components,10
,init,None
,solver,'cd'
,beta_loss,'frobenius'
,tol,0.0001
,max_iter,200
,random_state,42
,alpha_W,0.0
,alpha_H,'same'
,l1_ratio,0.0
,verbose,0


In [9]:
texts = [doc.split() for doc in cleaned_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Extract top words per topic
lda_topics = []
for topic in lda.components_:
    words = [count_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]]
    lda_topics.append(words)

coherence_model_lda = CoherenceModel(topics=lda_topics, texts=texts, dictionary=dictionary, coherence='c_v')
print("LDA Coherence Score:", coherence_model_lda.get_coherence())


LDA Coherence Score: 0.5757592723222666


In [10]:
lda_vis = pyLDAvis.prepare(
    topic_term_dists=lda.components_,
    doc_topic_dists=doc_topic_dists,
    vocab=count_vectorizer.get_feature_names_out(),
    doc_lengths=dtm.sum(axis=1).A1,
    term_frequency=dtm.sum(axis=0).A1
)
lda_vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.051702  0.102157       1        1  17.301407
7      0.233497  0.029087       2        1  15.130741
8     -0.042120  0.148272       3        1  14.299305
0     -0.048243 -0.103652       4        1   9.843872
1      0.131449 -0.013004       5        1   8.218584
6     -0.107442 -0.220397       6        1   7.636064
4      0.136181 -0.015566       7        1   7.522263
3     -0.127606 -0.048356       8        1   7.470510
9     -0.166714  0.150290       9        1   7.334479
5      0.042699 -0.028830      10        1   5.242776, topic_info=          Term          Freq         Total Category  logprob  loglift
31674  windows  1.695214e+08  1.695214e+08  Default  30.0000  30.0000
21700   people  5.026567e+08  5.026567e+08  Default  29.0000  29.0000
10038    drive  1.463557e+08  1.463557e+08  Default  28.0000  28.0000
29070    think  4.018324e+08  4.018324e+08  Default  27.0000  27.0000
5824      card  1.089817e+08  1.089817e+08  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
19914     need  5.699579e+06  1.894588e+08  Topic10   4.7645  11.0480
31852     work  5.500952e+06  1.959394e+08  Topic10   4.7290  10.9789
11128     even  5.466071e+06  2.701137e+08  Topic10   4.7227  10.6515
31904    would  5.736567e+06  7.358492e+08  Topic10   4.7710   9.6976
26538    since  5.215063e+06  1.651028e+08  Topic10   4.6757  11.0968

[815 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
26         1  0.015004   00pm
26         2  0.013122   00pm
26         3  0.012401   00pm
26         4  0.008537   00pm
26         5  0.007128   00pm
...      ...       ...    ...
32418      6  0.066337  years
32418      7  0.015959  years
32418      8  0.036105  years
32418      9  0.010868  years
32418     10  0.010474  years

[6810 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 9, 1, 2, 7, 5, 4, 10, 6])

In [11]:
# Save models
with open("lda_model.pkl", "wb") as f:
    pickle.dump(lda, f)

with open("nmf_model.pkl", "wb") as f:
    pickle.dump(nmf, f)

# Save LDA visualization as HTML
pyLDAvis.save_html(lda_vis, "lda_dashboard.html")
print("Models and visualization saved!")


Models and visualization saved!


In [12]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(cleaned_docs)
topic_model.visualize_topics()


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
